# Object Classification with OpenCV
OpenCV is an open source software library/toolbox with APIs in many programming languages.
It contains a collection of tools that are state-of-the-art in image processing and computer vision.

OpenCV 3.3.0 come with a highly improved deep learning module(`cv2.dnn`).
This module now supports a number of deep learning frameworks, including Caffe, TensorFlow, and Torch/PyTorch.
These frameworks provide pre-trained deep learning models so you can simply load a model from disk, pre-process an input image, and pass the image through the network and obtain the output classifications.

spiecifically for for image classification by using GoogLeNet trained network from Caffe model zoo.

[ImageNet](http://www.image-net.org/)


`python deep_learning_with_opencv.py --image images/jemma.png --prototxt bvlc_googlenet.prototxt --model bvlc_googlenet.caffemodel --labels synset_words.txt`

`python deep_learning_with_opencv.py --image src/deep-learning-opencv/images/jemma.png --prototxt src/deep-learning-opencv/bvlc_googlenet.prototxt --model src/deep-learning-opencv/bvlc_googlenet.caffemodel --labels src/deep-learning-opencv/synset_words.txt`

* `--image` : The path to the input image
* `--prototxt` : The path to the Caffe “deploy” prototxt file
* `--model` : The pre-trained Caffe model (i.e,. the network weights themselves)
* `--labels` : The path to ImageNet labels (i.e., “syn-sets”)

In [1]:
# import the necessary packages
import numpy as np
import time
import cv2

# load the input image from disk
image = cv2.imread("src/deep-learning-opencv/images/jemma.png")

# load the class labels from disk
rows = open("src/deep-learning-opencv/synset_words.txt")
classes = [r[r.find(" ") + 1:].split(",")[0] for r in rows]

# our CNN requires fixed spatial dimensions for our input image(s)
# so we need to ensure it is resized to 224x224 pixels while
# performing mean subtraction (104, 117, 123) to normalize the input;
# after executing this command our "blob" now has the shape:
# (1, 3, 224, 224)
blob = cv2.dnn.blobFromImage(image, 1, (224, 224), (104, 117, 123))

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe("src/deep-learning-opencv/bvlc_googlenet.prototxt", \
                               "src/deep-learning-opencv/bvlc_googlenet.caffemodel")

[INFO] loading model...


In [2]:
# set the blob as input to the network and perform a forward-pass to obtain our output classification
net.setInput(blob)
start = time.time()
preds = net.forward()
end = time.time()
print("[INFO] classification took {:.5} seconds".format(end - start))

[INFO] classification took 0.06398 seconds


In [3]:
# sort the indexes of the probabilities in descending order (higher probabilitiy first)
# and grab the top-5 predictions
idxs = np.argsort(preds[0])[::-1][:5]

In [4]:
# loop over the top-5 predictions and display them
for (i, idx) in enumerate(idxs):
    # draw the top prediction on the input image
    if i == 0:
        text = "Label: {}, {:.2f}%".format(classes[idx], preds[0][idx] * 100)
        cv2.putText(image, text, (5, 25),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # display the predicted label + associated probability to the console
    print("[INFO] {}. label: {}, probability: {:.5}".format(i + 1, classes[idx], preds[0][idx]))

[INFO] 1. label: beagle
, probability: 0.81137
[INFO] 2. label: Labrador retriever
, probability: 0.031416
[INFO] 3. label: bluetick
, probability: 0.023929
[INFO] 4. label: EntleBucher
, probability: 0.017507
[INFO] 5. label: Greater Swiss Mountain dog
, probability: 0.01444


In [5]:
# display the output image
cv2.imshow("Image", image)
cv2.waitKey(0)

113

## Script
Here is a Python script that is the equalivent to the code above.

```python
# USAGE
# python deep_learning_with_opencv.py --image images/jemma.png --prototxt bvlc_googlenet.prototxt --model bvlc_googlenet.caffemodel --labels synset_words.txt  #noqa

# import the necessary packages
import numpy as np
import argparse
import time
import cv2

# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True, help="path to input image")
ap.add_argument("-p", "--prototxt", required=True, help="path to Caffe 'deploy' prototxt file")       #noqa
ap.add_argument("-m", "--model", required=True, help="path to Caffe pre-trained model")               #noqa
ap.add_argument("-l", "--labels", required=True, help="path to ImageNet labels (i.e., syn-sets)")     #noqa
args = vars(ap.parse_args())

# load the input image from disk
image = cv2.imread(args["image"])

# load the class labels from disk
rows = open(args["labels"]).read().strip().split("\n")
classes = [r[r.find(" ") + 1:].split(",")[0] for r in rows]

# our CNN requires fixed spatial dimensions for our input image(s)
# so we need to ensure it is resized to 224x224 pixels while
# performing mean subtraction (104, 117, 123) to normalize the input;
# after executing this command our "blob" now has the shape:
# (1, 3, 224, 224)
blob = cv2.dnn.blobFromImage(image, 1, (224, 224), (104, 117, 123))

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

# set the blob as input to the network and perform a forward-pass to
# obtain our output classification
net.setInput(blob)
start = time.time()
preds = net.forward()
end = time.time()
print("[INFO] classification took {:.5} seconds".format(end - start))

# sort the indexes of the probabilities in descending order (higher
# probabilitiy first) and grab the top-5 predictions
idxs = np.argsort(preds[0])[::-1][:5]

# loop over the top-5 predictions and display them
for (i, idx) in enumerate(idxs):
    # draw the top prediction on the input image
    if i == 0:
        text = "Label: {}, {:.2f}%".format(classes[idx], preds[0][idx] * 100)
        cv2.putText(image, text, (5, 25),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)              #noqa

    # display the predicted label + associated probability to the console
    print("[INFO] {}. label: {}, probability: {:.5}".format(i + 1, classes[idx], preds[0][idx]))       #noqa

# display the output image
cv2.imshow("Image", image)
cv2.waitKey(0)
```

## Sources
* [Deep Learning with OpenCV](http://www.pyimagesearch.com/2017/08/21/deep-learning-with-opencv/)